In [ ]:
import os
import gc
import pandas as pd
import warnings
from sklearn.model_selection import train_test_split
import tensorflow as tf
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
DATA_DIRECTORY = ""

In [ ]:
train = pd.read_csv(os.path.join(DATA_DIRECTORY, 'train.csv'))
test = pd.read_csv(os.path.join(DATA_DIRECTORY, 'test.csv'))
labels = pd.read_csv(os.path.join(DATA_DIRECTORY, 'labels.csv'))

In [ ]:
test_id = test['SK_ID_CURR']
feature_list = list(train.columns)
train['TARGET'] = labels
train, val = train_test_split(train, test_size=0.2)
del labels
gc.collect()

In [ ]:
def dataframe_to_dataset(dataframe):
    dataframe = dataframe.copy()
    labels = dataframe.pop('TARGET')
    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
    ds = ds.shuffle(buffer_size=len(dataframe))
    return ds

In [ ]:
train_ds = dataframe_to_dataset(train)
val_ds = dataframe_to_dataset(val)
del train, val
gc.collect()

all_inputs = []

# Numeric features.
for header in feature_list:
  numeric_col = tf.keras.Input(shape=(1,), name=header)
  all_inputs.append(numeric_col)

all_features = tf.keras.layers.concatenate(all_inputs)
x = tf.keras.layers.Dense(256, activation="relu")(all_features)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(64, activation="relu")(x)
x = tf.keras.layers.Dropout(0.1)(x)
x = tf.keras.layers.Dense(16, activation="relu")(x)
x = tf.keras.layers.Dropout(0.5)(x)
output = tf.keras.layers.Dense(1)(x)
model = tf.keras.Model(all_inputs, output)

In [ ]:
model = tf.keras.Sequential([
  tf.keras.layers.InputLayer(input_shape=(len(feature_list),)),
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dropout(0.1),
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=[tf.keras.metrics.AUC()])

In [ ]:
model.fit(train_ds, epochs=10, validation_data=val_ds)

In [ ]:
tf.convert_to_tensor(test)

In [ ]:
predictions = model.predict(test)
submission = pd.DataFrame({'SK_ID_CURR': test_id.values, 'TARGET': predictions})
submission.to_csv('tf_nn.csv', index = False)